In [3]:
from lyft_dataset_sdk.utils.data_classes import LidarPointCloud
from lyft_dataset_sdk.lyftdataset import LyftDataset, LyftDatasetExplorer, Quaternion, view_points
import pandas as pd
import os
import numpy as np
from pathlib import Path

ROOT = "./train_data/"
POINTCLOUD = ROOT + "pointcloud/"
ANNOTATION = ROOT + "annotation/"
DATA_PATH = './3d-object-detection-for-autonomous-vehicles/'
lyftdata = LyftDataset(data_path=DATA_PATH, json_path=DATA_PATH+'train_data')
train = pd.read_csv('./3d-object-detection-for-autonomous-vehicles/train.csv')

9 category,
18 attribute,
4 visibility,
18421 instance,
10 sensor,
148 calibrated_sensor,
177789 ego_pose,
180 log,
180 scene,
22680 sample,
189504 sample_data,
638179 sample_annotation,
1 map,
Done loading in 43.7 seconds.
Reverse indexing ...
Done reverse indexing in 4.4 seconds.


In [36]:
def create_folder(path):
    try: 
        os.makedirs(path, exist_ok = True) 
    except OSError as error: 
        print("Directory '%s' can not be created" % path) 

def save_pointcloud(path, pd):
    np.save(path, pd)

def load_pointcloud(path):
    return np.load(path)

def save_annotation(path, at):
    with open(path, "w") as f:
        f.write(at)

def load_annotation(path):
    with open(path) as f:
        out = str(f.read().strip())
    return out

def extract_pd_at(idx, train = train, lyftdata = lyftdata):
    # Get the annotation
    label = train.iloc[idx]['PredictionString']
    
    # Get the pointcloud
    token = train.iloc[idx]['Id']
    sample_ = lyftdata.get('sample', token)
    
    output = np.array([1])
    if 'LIDAR_TOP' in sample_['data']:
        lidar_top = lyftdata.get('sample_data', sample_['data']['LIDAR_TOP'])
        pc = LidarPointCloud.from_file(Path("3d-object-detection-for-autonomous-vehicles/"+lidar_top['filename']))
        pc_top = pc.points[0:-1,:]
        output = pc_top
    if 'LIDAR_FRONT_RIGHT' in sample_['data']:
        lidar_front_right = lyftdata.get('sample_data', sample_['data']['LIDAR_FRONT_RIGHT'])
        pc = LidarPointCloud.from_file(Path("3d-object-detection-for-autonomous-vehicles/"+lidar_top['filename']))
        pc_front_right = pc.points[0:-1,:]
        if output.size <= 1:
            output = pc_front_right
        else:
            output = np.concatenate([output, pc_front_right], axis = 1)
    if 'LIDAR_FRONT_LEFT' in sample_['data']:
        lidar_front_left = lyftdata.get('sample_data', sample_['data']['LIDAR_FRONT_LEFT'])
        pc = LidarPointCloud.from_file(Path("3d-object-detection-for-autonomous-vehicles/"+lidar_top['filename']))
        pc_front_left = pc.points[0:-1,:]
        if output.size <= 1:
            output = pc_front_left
        else:
            output = np.concatenate([output, pc_front_left], axis = 1)
    return label, output

def save_pd_at(train = train):
    total_num = train.shape[0]
    for idx in range(total_num):
        if idx % 250 == 0 or idx == total_num - 1:
            print("Finish %4.2f%%" % ((idx/total_num)*100))
        annotation, pointcloud = extract_pd_at(idx)
        pointcloud_path = POINTCLOUD + "pointcloud_" + str(idx) + ".npy"
        annotation_path = ANNOTATION + "annotation_" + str(idx) + ".txt"
        save_pointcloud(pointcloud_path, pointcloud)
        save_annotation(annotation_path, annotation)

In [37]:
create_folder(ROOT)
create_folder(POINTCLOUD)
create_folder(ANNOTATION)

In [ ]:
save_pd_at()

Finish 0.00%
Finish 1.10%
Finish 2.20%
Finish 3.31%
Finish 4.41%
Finish 5.51%
